In [1]:

import polars as pl

vets = (
    pl
        .read_parquet("../data/dim_vet.parquet")
        .select(pl.all().shrink_dtype())

)

vets

vet,name,practice
i8,str,str
1,"""Zooey Codewell""","""TechSavvy Vets"""
2,"""Script Safari""","""Digital Wildlife Care"""
3,"""Debug Dino""","""Digital Wildlife Care"""
4,"""Pixel Paws""","""CodePaws Veterinary Clinic"""
5,"""Byte Lyon""","""TechSavvy Vets"""


In [2]:
batch_measurements = (
    pl
        .read_parquet("../data/batch_measurements.parquet")
        .select(pl.all().shrink_dtype())


)

batch_measurements

name,vet,age,weight,daily_steps,timestamp,vet_health_check,life_stage
str,i8,i8,f32,i16,datetime[μs],str,str
"""Arctic Archie""",3,25,579.503906,15306,2020-04-20 06:15:00,"""INJURED""","""SENIOR"""
"""Chilly Willy""",2,5,508.05246,6308,2020-04-20 06:15:00,"""INJURED""","""ADULT"""
"""Blizzard Bob""",5,35,867.150879,17702,2020-04-20 06:15:00,"""INJURED""","""SENIOR"""
"""Arctic Archie""",3,25,609.372009,6710,2020-04-22 06:15:00,"""INJURED""","""SENIOR"""
"""Chilly Willy""",3,5,482.300385,13696,2020-04-22 06:15:00,"""HEALTHY""","""ADULT"""
…,…,…,…,…,…,…,…
"""Peter Panda""",3,2,314.35965,8739,2024-04-17 06:15:00,"""SICK""","""JUV"""
"""Arctic Archie""",3,29,600.072205,13109,2024-04-17 06:15:00,"""SICK""","""SENIOR"""
"""Chilly Willy""",2,9,481.11673,1680,2024-04-17 06:15:00,"""HEALTHY""","""ADULT"""


In [3]:

# - How many times has vet practice Digital Wildlife Care diagnosed a polar bear as sick?

(
batch_measurements.join(vets, "vet")
    .filter(vet_health_check = "SICK", practice = "Digital Wildlife Care")
    .select(pl.len())
)

(
batch_measurements.filter(vet_health_check ="SICK")
.with_columns(pl.col("vet").cast(pl.Int64))
.filter(practice = "Digital Wildlife Care")
.join(dim_vet, on="vet")
.select(pl.len())
)


len
u32
553


In [4]:
# Which vet(s) has/have never seen a weight above the 99.9th percentile?

did_see = (
batch_measurements.select("weight", "practice").join(vets, "vet")
    .filter(pl.col("weight") > pl.col("weight").quantile(0.999))
    .select("name_right", "vet")
)

vets.join(did_see,"vet", how = "anti")

vet,name,practice
i8,str,str
1,"""Zooey Codewell""","""TechSavvy Vets"""
2,"""Script Safari""","""Digital Wildlife Care"""
3,"""Debug Dino""","""Digital Wildlife Care"""


In [35]:
# Which vet was the least consistent in name capitalization?
(
batch_measurements
    .select((pl.col("name").str.to_uppercase() == pl.col("name")).alias("isUpper"), "vet")
    .select("vet", pl.col("isUpper").sum().over("vet").alias("total_upper"),  pl.col("isUpper").count().over("vet").alias("total"))
)

inconsistent = (
batch_measurements
    .select((pl.col("name").str.to_uppercase() == pl.col("name")).sum().over("vet").alias("total_upper"), pl.col("name").count().over("vet").alias("total"), "vet", "timestamp")
    .filter(timestamp = pl.col("timestamp").min().over("vet"))
    .with_columns((pl.col("total_upper") / pl.col("total") - 0.5).abs().alias("test"))
    .filter(pl.col("test") == pl.col("test").min())
    .select((pl.col("test") + 0.5).alias("consistency"), "vet")


)

.select("name_right").unique
inconsistent.join(vets, on="vet")

consistency,vet,name,practice
f64,i8,str,str
0.505263,5,"""Byte Lyon""","""TechSavvy Vets"""
